In [826]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import io
import re
from datetime import date, timedelta
from dateutil.parser import parse
import numpy as np

In [822]:
from scraping_tools import *

In [831]:
URL = 'https://zkm.de/en/exhibitions-events/current-exhibitions'
example_title = 'Writing the History of the Future'

In [824]:
URL = 'https://www.microsoft.com/de-de/techwiese/events/default.aspx?Veranstalter=Community'
example_title = 'Bau dir einen Serverless EventStore'

In [825]:
URL = 'https://mlh.io/seasons/eu-2019/events'
example_title = 'HackSheffield'

# Start

In [834]:
raw_html = simple_get(URL)
soup = BeautifulSoup(raw_html, 'html.parser')

# Find the div-class of the example title
for string in [example_title, example_title.upper(), example_title.lower(), 
            example_title.title(), example_title.capitalize()]:

    try:
        example = soup.find(text=string)
    except:
        example = None
    if example:
        break

if example:
    ex = example
    for i in range(5):
        try:
            example_class = ex['class']
            break
        except:
            ex = ex.parent
    title_pos = np.where(example_title == np.array(ex.text.split('\n')))[0][0]
else:
    example_class = None

# Find the titles
matches = soup.find_all(attrs={'class': example_class})
titles = []
for match in matches:
    if match:
        m = match.text.split('\n')[title_pos]
        titles.append(m)
    else:
        titles.append('')

In [861]:
title = titles[0]
div = soup.find(text=title)
div = div.parent
re_link = 'href=\".*\"'
str(div)

'<a href="/en/exhibition/2017/10/open-codes">Open Codes</a>'

In [862]:
re.search(re_link, str(div)).group()

'href="/en/exhibition/2017/10/open-codes"'